In [3]:
!pip uninstall spacy cymem murmurhash preshed thinc blis -y
!pip install spacy
!pip install -numpy
#!spacy.prefer_gpu()
!pip install scispacy
!pip install pdfplumber
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
# !pip install pymupdf transformers torch torchvision torchaudio
!pip install https://s3.amazonaws.com/models.huggingface.co/bert/en_core_sci_md.tar.gz

Found existing installation: spacy 3.7.5
Uninstalling spacy-3.7.5:
  Successfully uninstalled spacy-3.7.5
Found existing installation: cymem 2.0.11
Uninstalling cymem-2.0.11:
  Successfully uninstalled cymem-2.0.11
Found existing installation: murmurhash 1.0.12
Uninstalling murmurhash-1.0.12:
  Successfully uninstalled murmurhash-1.0.12
Found existing installation: preshed 3.0.9
Uninstalling preshed-3.0.9:
  Successfully uninstalled preshed-3.0.9
Found existing installation: thinc 8.2.5
Uninstalling thinc-8.2.5:
  Successfully uninstalled thinc-8.2.5
Found existing installation: blis 0.7.11
Uninstalling blis-0.7.11:
  Successfully uninstalled blis-0.7.11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -n
  Using cached spacy-3.7.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
  Using cached blis-0.7.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached spacy-3.7.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
Using cached thinc-8.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (920 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.m

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz (14.8 MB)
  Preparing metadata (setup.py) ... done
  ERROR: HTTP error 404 while getting https://s3.amazonaws.com/models.huggingface.co/bert/en_core_sci_md.tar.gz
ERROR: Could not install requirement https://s3.amazonaws.com/models.huggingface.co/bert/en_core_sci_md.tar.gz because of HTTP error 404 Client Error: Not Found for url: https://s3.amazonaws.com/models.huggingface.co/bert/en_core_sci_md.tar.gz for URL https://s3.amazonaws.com/models.huggingface.co/bert/en_core_sci_md.tar.gz


In [19]:
!pip install requests
!pip install BeautifulSoup
!pip install langchain

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
import os
import requests
import xml.etree.ElementTree as ET
import spacy
import scispacy
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

In [2]:
nlp = spacy.load("en_core_sci_sm")

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
scibert_model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

/usr/local/lib/python3.11/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [26]:
import requests
import xml.etree.ElementTree as ET

# def fetch_arxiv_papers(domains, max_results=1):
#     all_papers = []
#     for domain in domains:
#         url = f"http://export.arxiv.org/api/query?search_query=cat:{domain}&start=0&max_results={max_results}"
#         response = requests.get(url)
#         if response.status_code != 200:
#             print(f"Error fetching data for domain: {domain}")
#             continue

#         root = ET.fromstring(response.text)
#         for entry in root.findall("{http://www.w3.org/2005/Atom}entry"):
#             title = entry.find("{http://www.w3.org/2005/Atom}title").text
#             summary = entry.find("{http://www.w3.org/2005/Atom}summary").text

#             # Extract the HTML link using the <link> tag with rel="alternate"
#             html_link = None
#             for link in entry.findall("{http://www.w3.org/2005/Atom}link"):
#                 if link.attrib.get('rel') == 'alternate':
#                     html_link = link.attrib.get('href')
#                     break

#             # Append the data to the list
#             all_papers.append({
#                 "Title": title.strip(),
#                 "Abstract": summary.strip(),
#                 "HTML_Link": html_link,
#                 "Domain": domain
#             })
#     return all_papers


import requests
from bs4 import BeautifulSoup

def extract_hrefs_from_url_by_title(url = 'https://arxiv.org/list/cs.AI/recent?skip=0&show=2000', target_title='View HTML'):
    try:
        response = requests.get(url)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        hrefs = []
        for a_tag in soup.find_all('a', title=target_title):
            if 'href' in a_tag.attrs:
                hrefs.append(a_tag['href'])
        return hrefs
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
        return []

def fetch_arxiv_ids(domains, max_results=5):
    """
    Fetches the ArXiv IDs of papers for the specified domains.

    Args:
        domains (list): A list of ArXiv subject categories (e.g., ["cs.AI", "physics.hep-th"]).
        max_results (int): The maximum number of results to fetch per domain (default: 100).

    Returns:
        list: A list of ArXiv paper IDs.
    """
    all_ids = []
    for domain in domains:
        url = f"http://export.arxiv.org/api/query?search_query=cat:{domain}&start=0&max_results={max_results}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error fetching data for domain: {domain}")
            continue

        root = ET.fromstring(response.text)
        for entry in root.findall("{http://www.w3.org/2005/Atom}entry"):
            # The ArXiv ID is typically found in the <id> tag.
            arxiv_id_full = entry.find("{http://www.w3.org/2005/Atom}id").text
            # The ID often looks like 'http://arxiv.org/abs/2304.01234v1'.
            # We want to extract just '2304.01234v1'.
            arxiv_id = arxiv_id_full.split('/')[-1]
            all_ids.append(arxiv_id)
    return all_ids


# Download PDF
def download_pdf(pdf_url, save_path="paper.pdf"):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
        return save_path
    return None


# Tokenization using SciSpaCy
def tokenize_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

# SciBERT Embedding
def get_scibert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = scibert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

In [21]:

import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import HTMLHeaderTextSplitter

def download_arxiv_html(arxiv_html_url, save_path):
    response = requests.get(arxiv_html_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    for script in soup(['script', 'style']):
        script.extract()

    cleaned_html = soup.prettify()

    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(cleaned_html)

    print(f"Saved HTML content to {save_path}")

def extract_abstract(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    abstract_div = soup.find('div', class_='ltx_abstract')
    abstract = abstract_div.get_text(strip=True) if abstract_div else ""
    if abstract_div:
        abstract_div.extract()  # remove abstract from soup
    return abstract, soup.prettify()

def split_html_by_heading(html_content):
    headers_to_split_on = [("h1", "Heading1"), ("h2", "Heading2"), ("h3", "Heading3")]
    splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    splits = splitter.split_text(html_content)

    main_paper = "\n".join(split.page_content for split in splits) if splits else ""
    return main_paper



In [43]:
# Example usage:
paper_types = ['cs.AI']
papers = extract_hrefs_from_url_by_title()
papers = papers[1:3]
print(papers)
for link in papers:
  save_as = 'paper.html'
  download_arxiv_html(link, save_as)

  with open(save_as, 'r', encoding='utf-8') as file:
      html_data = file.read()

  abstract, html_without_abstract = extract_abstract(html_data)
  main_paper = split_html_by_heading(html_without_abstract)

  print(abstract)
  print("============================")
  print("============================")
  print("============================")
  print("============================")
  print("============================")
  print("============================")
  print("============================")
  print("============================")
  print("============================")
  print("\nMain Paper:\n", main_paper[:10000])



['https://arxiv.org/html/2504.05278v1', 'https://arxiv.org/html/2504.05259v1']
Saved HTML content to paper.html
AbstractThis study investigates uncertainty quantification in large language models (LLMs) for medical applications, emphasizing both technical innovations and philosophical implications. As LLMs become integral to clinical decision-making, accurately communicating uncertainty is crucial for ensuring reliable, safe, and ethical AI-assisted healthcare. Our research frames uncertainty not as a mere impediment but as an inherent aspect of knowledge that invites a dynamic and reflective approach to AI design. By integrating advanced probabilistic methods—such as Bayesian inference, deep ensembles, and Monte Carlo dropout—with linguistic analysis that computes predictive and semantic entropy, we propose a comprehensive framework that differentiates and manages both epistemic and aleatoric uncertainties. The framework further incorporates surrogate modeling to circumvent the limita

In [15]:
#HAN + basic encoder decoder
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class HAN(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256, num_layers=2, output_dim=128):
        super(HAN, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        # x shape: (batch, seq_len, input_dim)
        h, _ = self.gru(x)  # (batch, seq_len, hidden_dim*2)

        # Attention
        attn_weights = torch.softmax(self.attention(h), dim=1)  # (batch, seq_len, 1)
        context = torch.sum(attn_weights * h, dim=1)  # (batch, hidden_dim*2)

        return self.fc(context)  # (batch, output_dim)

class BasicLSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, x):
        # x shape: (batch, seq_len, input_dim)
        _, (hidden, cell) = self.lstm(x)
        return hidden, cell

class BasicLSTMDecoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(output_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden, cell):
        # x shape: (batch, 1, output_dim) - for step-by-step decoding
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        return self.fc(output), hidden, cell

In [16]:
def get_han_embeddings(model, sentences):
    """Convert sentences to HAN embeddings"""
    tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
    scibert = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

    # Get SciBERT embeddings for each sentence
    sentence_embeddings = []
    for sent in sentences:
        inputs = tokenizer(sent, return_tensors="pt", truncation=True, max_length=128)
        with torch.no_grad():
            outputs = scibert(**inputs)
        sentence_embeddings.append(outputs.last_hidden_state.mean(dim=1))

    # Stack and pass through HAN
    embeddings = torch.stack(sentence_embeddings)  # (num_sents, 1, 768)
    return model(embeddings)  # (num_sents, output_dim)

def select_top_k(embeddings, sentences, model,k=5):
    """Select most important sentences using attention weights"""
    # Get attention scores (from HAN's attention layer)
    attn_scores = model.attention(embeddings).squeeze()  # (num_sents,)
    top_indices = torch.topk(attn_scores, k=k).indices
    return [sentences[i] for i in top_indices], embeddings[top_indices]

In [17]:
# Attention based encoder-decoder
class AttentionDecoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(output_dim, hidden_dim, num_layers, batch_first=True)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=8, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, encoder_outputs, hidden=None):
        # x shape: (batch, 1, output_dim)
        lstm_out, hidden = self.lstm(x, hidden)

        # Attention over encoder outputs
        attn_out, _ = self.attention(
            query=lstm_out,
            key=encoder_outputs,
            value=encoder_outputs
        )
        return self.fc(attn_out), hidden

In [18]:
class PaperDataset():
    def __init__(self, papers, tokenizer, max_len=512):
        self.papers = papers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        paper = self.papers[idx]
        inputs = self.tokenizer(
            " ".join(paper["selected_sentences"]),  # From HAN output
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        targets = self.tokenizer(
            paper["abstract"],
            max_length=150,  # Shorter for abstracts
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze()
        }

def train_model(model, dataloader, epochs=5):
  optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
  criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
  for epoch in range(epochs):
      model.train()
      total_loss = 0

      for batch in dataloader:
          optimizer.zero_grad()

          # Forward pass
          outputs = model(
              input_ids=batch["input_ids"].to(device),
              attention_mask=batch["attention_mask"].to(device),
              labels=batch["labels"].to(device)
          )

          loss = outputs.loss
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")